In [ ]:
import tensorflow as tf 
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split, StratifiedKFold
from scipy.stats import chi2_contingency 
from pipelines import preprocessor, preprocessor2
from preprocess import extra_features, separate
import pickle
import joblib

: 

In [ ]:
train= pd.read_csv("Datasets/train.csv")
train=extra_features(train)
separate(train)

: 

In [ ]:
X=train.drop('cost_category', axis=1)
y=train['cost_category']

: 

In [ ]:
X.shape

: 

In [ ]:
X.head(2)

: 

In [ ]:
col_drop=[]

for col in X.columns:
    
    ct=pd.crosstab(y, X[col])
    chi2, P, _ , _ = chi2_contingency(ct)
    
    if P >= 0.05:
        col_drop.append(col)
    elif chi2 <= 1500:
            col_drop.append(col)
        
    print(f"Column: {col}\nChi_Test: {chi2}\nP_Value: {P}\n*************")

: 

In [ ]:
col_drop

: 

In [ ]:
X.drop(columns=col_drop, inplace=True)
X.shape

: 

In [ ]:
X= preprocessor(X)
X

: 

In [ ]:
from sklearn.preprocessing import LabelEncoder

le=LabelEncoder()
y=le.fit_transform(y)
mapping= dict(zip(le.classes_, le.transform(le.classes_)))
mapping

: 

In [ ]:
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.2, shuffle=True, stratify= y, random_state=1)

print(f"X_train: {X_train.shape}\nX_test: {X_test.shape}\ny_train: {y_train.shape}\ny_test: {y_test.shape}")

: 

In [ ]:
n_features= X.shape[1]
num_classes= len(le.classes_)

model= Sequential()
model.add(Dense(32, input_dim=n_features, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))

epochs=30
learning_rate= 0.01
model.compile(loss = 'sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

: 

In [ ]:
model.fit(X_train, y_train, epochs= epochs, batch_size=64, verbose=1)

: 

In [ ]:
loss, accuracy= model.evaluate(X_test, y_test, verbose=1)
print(f"Accuracy Score: {accuracy }\nLoss: {loss}")

: 

In [ ]:
folds= StratifiedKFold(n_splits=10, shuffle=True, random_state=42 )

for fold, (train_idx, val_idx) in enumerate (folds.split(X,y)):
    X_train, X_test = X[train_idx], X[val_idx]
    y_train, y_test = y[train_idx], y[val_idx]

: 

In [ ]:
model.fit(X_train, y_train, epochs=epochs, batch_size=64, verbose=1)

: 

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test, verbose=1)
print(f"Accuracy Score: {accuracy }\nLoss: {loss}")

: 